In [4]:
# === TOPIX Core 30 の株価(1y,1d)を取得して Parquet に保存 + manifest更新 + S3アップロード ===
from pathlib import Path
import os
import json
import hashlib
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import yfinance as yf
# ファイル先頭付近（importの後）に追加
from dotenv import load_dotenv
from pathlib import Path

for p in (Path(".env.s3"), Path(".env")):
    if p.exists():
        load_dotenv(dotenv_path=p, override=False)
# ---- 入出力 ----
PARQUET_DIR   = Path("./data/parquet")
WEIGHT_PARQUET = PARQUET_DIR / "topixweight_j.parquet"
OUT_PRICES     = PARQUET_DIR / "core30_prices_1y_1d.parquet"  # Dashが読み込む価格テーブル
OUT_META       = PARQUET_DIR / "core30_meta.parquet"          # {code, stock_name, ticker}
MANIFEST_PATH  = PARQUET_DIR / "manifest.json"

# ---- S3 設定（存在すればアップロードを実行）----
DATA_BUCKET     = os.getenv("DATA_BUCKET")                     # 例: "dash-plotly"
PARQUET_PREFIX  = os.getenv("PARQUET_PREFIX", "parquet/")      # 例: "parquet/"
AWS_REGION      = os.getenv("AWS_REGION")
AWS_PROFILE     = os.getenv("AWS_PROFILE")   # ~/.aws/credentials を使う場合に指定

# ---- 便利関数 ----
def _sha256_of(path: Path, chunk_size: int = 1024 * 1024) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        while True:
            b = f.read(chunk_size)
            if not b:
                break
            h.update(b)
    return h.hexdigest()

def _write_manifest_atomic(items: list[dict], path: Path) -> None:
    """
    items: [{"key": "core30_meta.parquet", "bytes": 123, "sha256": "...", "mtime": "...Z"}, ...]
    """
    path.parent.mkdir(parents=True, exist_ok=True)
    payload = {
        "generated_at": datetime.now(timezone.utc).isoformat(),
        "items": sorted(items, key=lambda d: d["key"]),
        "note": "Auto-generated. Do not edit by hand."
    }
    tmp = path.with_suffix(path.suffix + ".tmp")
    with tmp.open("w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    tmp.replace(path)

def _maybe_upload_to_s3(files: list[Path]) -> None:
    if not DATA_BUCKET:
        print("[INFO] DATA_BUCKET 未設定のため S3 アップロードはスキップします。")
        return
    try:
        import boto3
        session_kwargs = {}
        if AWS_PROFILE:
            session_kwargs["profile_name"] = AWS_PROFILE
        session = boto3.Session(**session_kwargs) if session_kwargs else boto3.Session()
        s3 = session.client("s3", region_name=AWS_REGION) if AWS_REGION else session.client("s3")
    except Exception as e:
        print(f"[WARN] boto3 初期化に失敗: {e}  S3アップロードをスキップします。")
        return

    for p in files:
        key = f"{PARQUET_PREFIX}{p.name}"
        try:
            extra = {
                "ServerSideEncryption": "AES256",
                "CacheControl": "max-age=60",
                "ContentType": "application/octet-stream",
            }
            s3.upload_file(str(p), DATA_BUCKET, key, ExtraArgs=extra)
            print(f"[OK] uploaded: s3://{DATA_BUCKET}/{key}")
        except Exception as e:
            print(f"[WARN] upload failed for {p} → s3://{DATA_BUCKET}/{key}: {e}")

# ---- 読み込み（Core30 抽出）----
if not WEIGHT_PARQUET.exists():
    raise FileNotFoundError(f"not found: {WEIGHT_PARQUET}")

w = pd.read_parquet(WEIGHT_PARQUET, engine="pyarrow")

# 安全化
for col in ("code", "stock_name", "size_class"):
    if col not in w.columns:
        raise KeyError(f"required column missing: {col}")

w["code"] = w["code"].astype("string")
w["size_class"] = w["size_class"].astype("string")

# "TOPIX Core30" と "TOPIX Core 30" の両方に耐性（空白を除去して包含判定）
_mask_core30 = w["size_class"].str.replace(" ", "", regex=False).str.contains("Core30", case=False, na=False)
core = (
    w.loc[_mask_core30, ["code", "stock_name"]]
     .drop_duplicates(subset=["code"])
     .reset_index(drop=True)
)

def _to_ticker(x: str) -> str:
    s = str(x).strip()
    return s if s.endswith(".T") else f"{s}.T"

core["ticker"] = core["code"].map(_to_ticker)

if core.empty:
    raise RuntimeError("Core30 list is empty. Check 'size_class' values in topixweight_j.parquet.")

tickers = core["ticker"].tolist()

# ---- yfinance 取得（1y,1d）----
def _flatten_multi(raw: pd.DataFrame, tickers: list[str]) -> pd.DataFrame:
    """
    yfinance.download(..., group_by='ticker') の MultiIndex 列を
    tidy形式 [date, ticker, Open, High, Low, Close, Volume] に整形。
    取得できなかった銘柄は自動スキップ。
    """
    frames = []
    if isinstance(raw.columns, pd.MultiIndex):
        # 典型: level=0 が ticker, level=1 が OHLCV
        for t in tickers:
            if t in raw.columns.get_level_values(0):
                sub = raw[t].copy()
                if sub.empty:
                    continue
                sub = sub.reset_index()
                # index名が "Date" / None などケースがあるので正規化
                if "Date" in sub.columns:
                    sub = sub.rename(columns={"Date": "date"})
                elif "index" in sub.columns:
                    sub = sub.rename(columns={"index": "date"})
                else:
                    # 念のため
                    sub.columns = ["date"] + [c for c in sub.columns[1:]]
                sub["ticker"] = t
                keep = [c for c in ["date","Open","High","Low","Close","Volume","ticker"] if c in sub.columns]
                frames.append(sub[keep])
    else:
        # 単一銘柄や想定外の形の場合：汎用処理
        sub = raw.reset_index()
        if "Date" in sub.columns:
            sub = sub.rename(columns={"Date": "date"})
        elif "index" in sub.columns:
            sub = sub.rename(columns={"index": "date"})
        sub["ticker"] = tickers[0] if tickers else "UNKNOWN"
        keep = [c for c in ["date","Open","High","Low","Close","Volume","ticker"] if c in sub.columns]
        frames.append(sub[keep])

    if not frames:
        return pd.DataFrame(columns=["date","Open","High","Low","Close","Volume","ticker"])
    out = pd.concat(frames, ignore_index=True)
    # tz-aware → naive への安全化
    if np.issubdtype(out["date"].dtype, np.datetime64):
        try:
            out["date"] = pd.to_datetime(out["date"]).dt.tz_localize(None)
        except Exception:
            out["date"] = pd.to_datetime(out["date"], utc=True).dt.tz_localize(None)
    else:
        out["date"] = pd.to_datetime(out["date"], errors="coerce")
    # 列型を最小限整える
    for c in ["Open","High","Low","Close","Volume"]:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

# まとめてダウンロード（失敗時は個別フォールバック）
try:
    raw = yf.download(
        tickers,
        period="1y",
        interval="1d",
        group_by="ticker",
        threads=True,
        progress=False,
    )
    prices = _flatten_multi(raw, tickers)
    if prices.empty:
        raise RuntimeError("yf.download returned empty. fallback to per-ticker.")
except Exception:
    frames = []
    for t in tickers:
        try:
            r = yf.download(t, period="1y", interval="1d", group_by="ticker", threads=True, progress=False)
            f = _flatten_multi(r, [t])
            if not f.empty:
                frames.append(f)
        except Exception:
            # 取得失敗ティッカーはスキップ（エラーでNotebook停止させない）
            pass
    prices = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

# 必須列がなければ空
need = {"date","Open","High","Low","Close","ticker"}
if prices.empty or not need.issubset(prices.columns):
    raise RuntimeError("No price data collected or required columns missing.")

# ---- メタ保存 & 価格保存 ----
OUT_PRICES.parent.mkdir(parents=True, exist_ok=True)
core.to_parquet(OUT_META, engine="pyarrow", index=False)
prices.to_parquet(OUT_PRICES, engine="pyarrow", index=False)

print(f"[OK] meta  saved: {OUT_META}   rows={len(core)}")
print(f"[OK] prices saved: {OUT_PRICES} rows={len(prices)}")

# ---- manifest.json を原子的に作成/更新 ----
items = []
for p in [OUT_META, OUT_PRICES]:
    stat = p.stat()
    items.append({
        "key": p.name,
        "bytes": stat.st_size,
        "sha256": _sha256_of(p),
        "mtime": datetime.fromtimestamp(stat.st_mtime, tz=timezone.utc).isoformat()
    })
_write_manifest_atomic(items, MANIFEST_PATH)
print(f"[OK] manifest updated: {MANIFEST_PATH}")

# ---- S3 へアップロード（環境変数があれば）----
_to_upload = [OUT_META, OUT_PRICES, MANIFEST_PATH]
_maybe_upload_to_s3(_to_upload)


[OK] meta  saved: data/parquet/core30_meta.parquet   rows=30
[OK] prices saved: data/parquet/core30_prices_1y_1d.parquet rows=7350
[OK] manifest updated: data/parquet/manifest.json
[OK] uploaded: s3://dash-plotly/parquet/core30_meta.parquet
[OK] uploaded: s3://dash-plotly/parquet/core30_prices_1y_1d.parquet
[OK] uploaded: s3://dash-plotly/parquet/manifest.json
